In [1]:
import pandas as pd
import praw
import secrets
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.svm import SVC
import matplotlib.pyplot as plt


# Class: Subreddit_Predictor

Objects of this class contain attributes and Methods that can be broken up into three categories: **Data**, **Collections**, and **Processing**

## Data
Pandas DataFrames and methods to update and clean the data.

**Attributes:**

| Name      |   Type    |             Description             |
|:----------|:---------:|:-----------------------------------|
| raw_data  | DataFrame |      The raw unprocessed data       |
| full_data | DataFrame |         The processed data          |
| subreddits |   list    | A list of subreddits. Extraced from full_data. |
| X_train   | pd.Series | the X portion of the training data  |
| Y_train   | np.array  | the Y portion of the training data  |
| X_test    | pd.Series |   the X portion of the test data    |
| Y_test    | np.array  |   the Y portion of the test data    |

**Methods:**

| Name (with input/output typing) | Description                                                                                                                               |
|---------------------------------| ------------------------------------------------------------------------------------------------------------------------------------------|
| add_data(df: DataFrame)         | Updates the raw_data attribute                                                                                                            |
| ready_data()                    | Cleans the data and does a test train split. <br/> Overwrites the full_data attribute. <br/> Creates the X_train, Y_train, X_test, and Y_test attributes. |

## Collections
Contains dictionaries of vectorizers, classifiers, and models

**Attributes:**

| Name        | Type                  | Description                                                                                                                                |
|:------------|:----------------------|:-------------------------------------------------------------------------------------------------------------------------------------------|
| Vectorizers | dict | Dictionary of Vectorizer objects                                                                                                           |
| Feature_Vectors | dict | Dictionary of the vectorized full_data |
| Classifiers | dict | Dictionary of Classifier objects                                                                                                           |
| Models      | dict | Dictionary of trained Classifier objects                                                                                                   |
| Models_info | dict        | Dictionary containing a description of each model in Models.                                                                               |
| Predictions | DataFrame   | A DataFrame with all the titles and actual subreddits in X_test and Y_test <br/> There is a column for each model that has the predictions |
| Results | dict | Dictionary of DataFrames for each model. Each row and column is a subreddit. Shows the number of false classifications |

**Methods:**

| Name (with input/output typing)                                                                                  | Description                                                                                                                                                                                |
|------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| add_vectorizer(model: Vectorizer)                                                                                | Trains the vectorizer. <br/>Adds (key = model.name, value = model) to Vectorizers                                                                                                               |
| add_feature_vectors(vectorizerName: str)| Embeds X_train as vectors and creates a dataframe for the new feature vectors. <br/> Adds this dataFrame to Feature_Vectors |
| add_classifier(model: Classifier)                                                                                | Adds (key = model.name, value = model) to Classifiers                                                                                                                                      |
| train_model(<br/>modelName: str, <br/>vectorizerName: str, <br/>classifierName: str, <br/>description = '' :str) | Takes vectorizer and classifer from Vectorizers and Classifiers. <br/>Trains the classifier.<br/>Names and adds the trained model to Models.<br/>Adds the description text to Models_info. |
| test_model(modelName: str) | Runs the model against X_test and Y_test. <br/> Updates Predictions and Results                                                                                                            |

## Processing

**Methods:**

| Name (with input/output typing)                       | Description                                                                                   |
|-------------------------------------------------------|-----------------------------------------------------------------------------------------------|
| predict(modelName:str, title: str, titles: iter[str]) | Given a model, enter a title or a list/dataframe of titles. Will return the model prediction. |
| compare(models: list[str])                            | Creates a bar chart comparing each of the models on each of the subreddits. |                  |

In [2]:
class Subreddit_Predictor:

    def __init__(self):

        #Data
        self.raw_data = pd.DataFrame({'id': [], 'title': [], 'subreddit': []})
        self.full_data = pd.DataFrame({'id': [], 'title': [], 'subreddit': []}).set_index('id')
        self.subreddits = []

        #Collections
        self.Vectorizers = {}
        self.Feature_Vectors = {}
        self.Classifiers = {}
        self.Models = {}
        self.Models_info = {}
        #self.Predictions = {}
        self.Results = {}

    #### Data ####

    def add_data(self, df):
        """df is a pandas DataFrame with columns={'title':[], 'subreddit':[]}. It will be merged with the existing raw_data"""
        self.raw_data = pd.concat([self.raw_data, df]).drop_duplicates(subset='id')

    def ready_data(self, test_size=.2, seed=42):
        """Splits and encodes the data. Saves is in X_train, Y_train, X_test, Y_test."""

        # Clean the data
        self._clean_data()

        # Update the subreddits attribute
        self.subreddits = self.full_data['subreddit'].unique().tolist()

        # Encode the subreddits
        self._le = LabelEncoder()
        self.full_data['subreddit_num'] = self._le.fit_transform(self.full_data['subreddit'])

        # Split the data
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(self.full_data['title'],
                                                                                self._le.fit_transform(
                                                                                    self.full_data['subreddit']),
                                                                                test_size=test_size, random_state=seed)
        # Record the indices
        self._train_index = self.X_train.index
        self._test_index = self.X_test.index

        # Update Predictions
        df = self.full_data
        df = df.drop(columns=['subreddit_num'])
        df = df.loc[obj._test_index]
        df = df.rename(columns={'subreddit': 'actual'})
        self.Predictions = df

    def _clean_data(self):
        """Cleans the data in raw_data and updates self.data"""

        df = self.raw_data

        # Remove all non-alpha-numeric characters
        df['title'] = df['title'].str.replace(r'[^a-zA-Z0-9 ]', '', regex=True)

        # Make all the text lowercase
        df['title'] = df['title'].str.lower()

        # Remove empty rows
        df['title'] = df['title'].str.strip()
        filter = df['title'] == ''
        df = df.drop(df[filter].index)

        # Store it as
        self.full_data = df

        # Change the index
        self.full_data = self.full_data.set_index('id')

    #### Collections ####

    def add_vectorizer(self, vectorizer):
        """This is how we add a vectorizers to our collection"""
        vectorizer.train(self.X_train)
        self.Vectorizers[vectorizer.vectorizerName] = vectorizer

    def add_feature_vectors(self, vectorizerName):
        """Updates Feature_Vectors"""
        vectorizer = self.Vectorizers[vectorizerName]
        self.Feature_Vectors[vectorizerName] = vectorizer.embed(self.full_data['title'])

    def add_classifier(self, classifier):
        """We add the classifier to our collection, self.Classifiers"""
        self.Classifiers[classifier.classifierName] = classifier

    def train_model(self, modelName, vectorizerName, classifierName, description=''):
        """
        :param modelName: The name of this model
        :param vectorizerName: Which feature vectors are we using?
        :param classifierName: Which classifier are we using?
        :param description: Write a short discription of the model (optional).
        :return: Adds a trained object of the classifier class to self.Models
        """

        self.Models_info[modelName] = {'vectorizerName': vectorizerName, 'classifierName': classifierName,
                                       'description': description}

        X_train = self.Feature_Vectors[vectorizerName].loc[self._train_index]
        Y_train = self.Y_train
        classifier = self.Classifiers[classifierName]
        classifier.train(X_train, Y_train)

        self.Models[modelName] = classifier

    def test_model(self, modelName):
        """Tests the model using X_test and Y_test. Updates Predictions and Results"""

        model = self.Models[modelName]

        vectorizerName = self.Models_info[modelName]['vectorizerName']
        X_test = self.Feature_Vectors[vectorizerName].loc[self._test_index]

        yhat = model.predict(X_test)
        yhat = self._le.inverse_transform(yhat)
        self.Predictions[modelName] = yhat

        df = self.Predictions
        results_matrix = pd.DataFrame(columns=self.subreddits)
        for actual_sub in self.subreddits:
            row = []
            filter = df['actual'] == actual_sub
            temp_df = df[filter]
            total = len(temp_df)
            for predicted_sub in self.subreddits:
                filter = temp_df[modelName] == predicted_sub
                count = len(temp_df[filter])
                row.append(count / total)

                #print('actual_sub', actual_sub, 'predicted_sub', predicted_sub)

            results_matrix.loc[actual_sub] = row

        self.Results[modelName] = results_matrix

    #### Processing ####

    def predict(self, modelName, titles):
        """
        :param modelName: Which model are we using?
        :param titles: A list or series of titles
        :return: A data frame of 'title' and 'prediction'
        """

        model = self.Models[modelName]

        vectorizerName = self.Models_info[modelName]['vectorizerName']
        vectorizer = self.Vectorizers[vectorizerName]

        title_vectors = vectorizer.embed(titles)

        array = model.predict(title_vectors)
        df = pd.DataFrame(array)
        df['title'] = titles
        df['prediction'] = self._le.inverse_transform(array)
        df = df.drop(columns=[0])
        return df

    def compare(self, list_of_models):
        """Creates a bar chart comparing the predictions from a list of models"""

        num_models = len(list_of_models)

        # Set the width of the bars
        bar_width = 1 / (num_models + 1)

        #Results = self.Results
        Results = {key:self.Results[key] for key in list_of_models}

        keys = list(list(Results.items())[0][1].keys())
        x_pos = [i for i, _ in enumerate(keys)]

        for key, val in Results.items():
            correct = []
            for sub in self.subreddits: correct.append(val.loc[sub][sub])
            plt.bar(x_pos, correct, bar_width, label=key)
            x_pos = [i + bar_width for i in x_pos]

        x_pos = [i - bar_width for i in x_pos]

        # Set the x-axis tick labels
        plt.xticks(x_pos, keys, rotation=45, ha='right')

        # Add a title and axis labels
        plt.title('Which Subreddits were easiest to predict?')
        plt.xlabel('Subreddit')
        plt.ylabel('% Correct')

        # Add a legend
        plt.legend()

        # Show the plot
        plt.show()



# Class: Vectorizer

Objects of this class are vectorizers, like Bag-of-Words or Doc2Vec. They have very few attributes and methods.
All of the attributes and methods will be overwritten by each object of this class.

**Attributes:**

| Name           | Type | Description                                                                                         |
|----------------|------|-----------------------------------------------------------------------------------------------------|
| vectorizerName | str | The name of this vectorizer. <br/> This will be the key for any dictionaries containing it.         |
| description    | str | A breif discription of what this vectorizer is/does. <br/>Put the parameters here if there are any. |
| model | Other | The actual model. Typically an object of a class like Gensim or SCM

**Methods:**

| Name                               | Description                                                                                          |
|------------------------------------|------------------------------------------------------------------------------------------------------|
| train(X_train: Series)             | Uses the training data to train the model.                                                           |
| embed(titles: Series) -> DataFrame | Takes a list/dataFrame of titles and returns a DataFrame of the embeddings for each of them. |



In [3]:
class Vectorizer:
    """This class is to hold all of the Title Vectorizers, like Bag-of-Words and Doc2Vec. Each vectorizer is a specific object. The class methods all have the same input/output."""

    def __init__(self, vectorizerName):
        self.vectorizerName = vectorizerName
        self.description = "Description goes here"

    def train(self, X_train):
        """Inputs the training data. Creates the self.model"""

        self.model = self._train(X_train)

    def _train(self, X_train):
        """Just a place holder for the actual function"""
        pass

    def embed(self, titles):
        """Given a data frame or series with only titles, will return a df of all of the features, indexed by id. The actual function will be added to each object."""

        df =  self._embed(titles, self.model)

        return df


    def _embed(self, titles, model):
        """Just a place holder for the actual function."""
        pass



# Class: Classifier

This class holds the classifiers, like XGBoost and Support Vector Machines.
It also has very few attributes and methods.

**Attributes:**

|Name | Type | Description |
|-----|-------|--------|
| classifierName | str | The name of this classifier |
| description | str | A brief description of this classifier |
| model | Other | Where the actual model is stored. Typically a member of a totally different class. |

**Methods:**

|Name | Description|
|-----| --------|
|train(X_train: pd.Series, Y_train: np.array) | Trains the model |
|predict(titles: pd.Series) | predicts where each title should go |

In [4]:
class Classifier:
    """This is the class the holds the classifiers"""

    def __init__(self, classifierName):
        self.classifierName = classifierName

    def train(self, X_train, Y_train):
        """Input the X and Y training data. Then update the model"""

        self.model = self._train(X_train, Y_train)

    def _train(self, X_train, Y_train):
        """Where the real function is stored"""
        pass

    def predict(self, title_vectors):
        """
        :param title_vectors: A pandas dataframe of the vectorized titles
        :return: A pandas series with the predictions
        """

        return self._predict(title_vectors, self.model)

    def _predict(self, titles, model):
        """where the actual function is stored"""
        pass
